# Projeto 4: Classificação binária brest cancer - classificar somente um registro e slavar o classificador

## Etapa 1: Importação das Bibliotecas

In [1]:
!pip install torch==1.4.0

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
torch.__version__

'1.4.0'

## Etapa 2: Base de dados

In [3]:
np.random.seed(123)
torch.manual_seed(123)

In [4]:
previsores = pd.read_csv('/content/entradas_breast.csv')
classes = pd.read_csv('/content/saidas_breast.csv')

In [5]:
previsores = torch.tensor(np.array(previsores), dtype=torch.float)
classes = torch.tensor(np.array(classes), dtype=torch.float)

In [6]:
previsores.shape, classes.shape

(torch.Size([569, 30]), torch.Size([569, 1]))

In [7]:
type(previsores), type(classes)

(torch.Tensor, torch.Tensor)

## Etapa 3: Transformação dos dados para tensores

In [8]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classes),
                                           batch_size = 10, shuffle = True)

## Etapa 4: Construção do modelo

In [9]:
class cls_torch(nn.Module):

  def __init__(self):
    super().__init__()

    # 30 -> 16 -> 16 -> 1                       ARQUITETURA DA REDE
    self.dense0 = nn.Linear(30,8)               # Ligação das entradas com a 1ª camada de neurônios

    # Mesmos parâmetors usados pelo kernel initializer do Keras [https://keras.io/initializers/, ver sessão RandomNormal]
    torch.nn.init.normal_(self.dense0.weight,
                          mean=0.0, std = 0.05) # Inicializa os pesos com uma distribuição estatística uniforme
    
    self.dense1 = nn.Linear(8,8)                # Ligação da 1º camada com a 2º camada de neurônios
    torch.nn.init.normal_(self.dense1.weight,
                          mean=0.0, std = 0.05) # Inicializa os pesos com uma distribuição estatística uniforme
    self.activation = nn.ReLU()                 # Função de ativação da 2ª camada
    
    self.dense2 = nn.Linear(8,1)                # Ligação da 2º camada com a camada de saída
    self.dropout = nn.Dropout(0.2)
    self.output = nn.Sigmoid()                  # Função de ativação da 3ª camada (Output)

  def forward(self, X):
    """
    Recebe as entradas e faz a transmição de uma camada para outra até a saída
    :param X: Entrada no formato Numpy Array
    :return: Saída das previsões (Probabilidades)
    """
    X = self.dense0(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense1(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

In [10]:
cls = cls_torch()

In [11]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(cls.parameters(), lr = 0.001,
                             weight_decay = 0.0001)

## Etapa 5: Treinamento do modelo

In [12]:
for epoch in range(100):
    running_loss = 0.
    
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()        

        outputs = cls(inputs)
        loss = criterion(outputs, labels)
        loss.backward()    
        optimizer.step()

        running_loss += loss.item()

    print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

Época   1: perda 1.50244
Época   2: perda 0.73373
Época   3: perda 0.67770
Época   4: perda 0.58107
Época   5: perda 0.57171
Época   6: perda 0.56428
Época   7: perda 0.53814
Época   8: perda 0.50038
Época   9: perda 0.50480
Época  10: perda 0.49681
Época  11: perda 0.47105
Época  12: perda 0.46919
Época  13: perda 0.46847
Época  14: perda 0.46689
Época  15: perda 0.43371
Época  16: perda 0.41159
Época  17: perda 0.40354
Época  18: perda 0.41222
Época  19: perda 0.39259
Época  20: perda 0.38620
Época  21: perda 0.37312
Época  22: perda 0.35878
Época  23: perda 0.36107
Época  24: perda 0.32681
Época  25: perda 0.33623
Época  26: perda 0.36341
Época  27: perda 0.32568
Época  28: perda 0.32805
Época  29: perda 0.31555
Época  30: perda 0.30389
Época  31: perda 0.31813
Época  32: perda 0.33555
Época  33: perda 0.32136
Época  34: perda 0.31885
Época  35: perda 0.29142
Época  36: perda 0.31004
Época  37: perda 0.30168
Época  38: perda 0.27757
Época  39: perda 0.30899
Época  40: perda 0.26122


## Etapa 6: Classificar somente um registro

In [13]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [14]:
cls.eval()

cls_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (activation): ReLU()
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [15]:
previsao = cls(novo)
previsao

tensor([[1.]], grad_fn=<SigmoidBackward>)

In [16]:
previsao = previsao.detach()
previsao

tensor([[1.]])

In [17]:
previsao = previsao.numpy()
type(previsao)

numpy.ndarray

In [18]:
previsao

array([[1.]], dtype=float32)

In [19]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

## Etapa 7: Salvar o classificador

In [20]:
cls.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-6.2938e-01, -6.3880e-02, -3.8812e-01, -1.0131e-01,  5.3613e-02,
                       -5.2737e-02,  5.9786e-02, -6.1688e-02,  2.9235e-02,  8.3439e-02,
                        5.2361e-04,  1.0154e-03,  6.2317e-04, -1.3866e-03,  1.9200e-02,
                        1.1764e-01, -3.8250e-01, -2.6600e-01, -6.0280e-03,  4.2274e-01,
                       -4.7407e-01, -3.1142e-02, -3.2840e-01,  1.8073e-01,  4.6571e-02,
                        4.2041e-02,  2.2236e-02,  4.5930e-02,  3.1673e-02,  6.3382e-01],
                      [ 8.2932e-02,  1.6685e-02,  5.3594e-02, -5.2358e-02,  8.3542e-02,
                       -2.1864e-04,  4.8437e-02, -1.6590e-03, -7.1115e-02, -2.8024e-02,
                        1.3979e-02, -1.1546e-02, -4.8893e-02,  1.0772e-01, -1.0195e-14,
                        1.0536e-04,  1.7714e-02, -3.6042e-13, -3.3771e-03, -4.1130e-14,
                       -5.3232e-02,  4.5065e-02, -4.9866e-02, -6.6553e-02, -8.3494e-02,


In [22]:
torch.save(cls.state_dict(), 'cls_breast_cancer.pth')